In [40]:
import web_scrapping
import db
import pandas as pd
from datetime import datetime

data = []
fecha_actual = datetime.now().strftime("%Y-%m-%d")

# Extraemos los productos y sus enlaces a la web de cada supermercado, y creamos un diccionario.     
consulta = "SELECT nombre_producto, link_coto, link_carrefour, link_dia FROM productos"
resultado = db.ejecutar_consulta(consulta)
productos = {}
for row in resultado:
    nombre_producto = row[0]
    enlaces = {
        "Coto": row[1],
        "Carrefour": row[2],
        "Día": row[3]
    }
    productos[nombre_producto] = enlaces

for producto, urls in productos.items():
    precios = {"producto": producto, "fecha": fecha_actual}
    for supermercado, url in urls.items():
        precio = None  # Inicializamos precio con el valor "null"
        try:
            if supermercado == "Coto":
                precio = web_scrapping.extraer_precio_coto(url)
            elif supermercado == "Carrefour":
                precio = web_scrapping.extraer_precio_carrefour(url)
            elif supermercado == "Día":
                precio = web_scrapping.extraer_precio_dia(url)
        except Exception as e:
            print(f"Error al extraer el precio del producto {producto} en {supermercado}: {e}")
        precios[supermercado] = precio if precio is not None else None
    data.append(precios)

df = pd.DataFrame(data)     # Convertimos a dataframe.
df.head()

Conexión cerrada correctamente.


,producto,fecha,Coto,Carrefour,Día
0,Azúcar Ledesma 1kg,2024-04-05,1399.0,1400.0,1395.0
1,Harina Morixe 0000 1kg,2024-04-05,870.0,Precio no encontrado,870.0
2,Harina Leudante Pureza 1 Kg,2024-04-05,1118.0,1120.0,1170.0
3,Arroz Integral Molinos Ala 500 Gr,2024-04-05,1519.0,1510.0,1510.0
4,Yerba Mate Playadito Suave 1 Kg,2024-04-05,4865.0,4105.0,3650.0


In [4]:
import db
import pandas as pd

try:
    consulta = '''SELECT DISTINCT ON (producto) producto, fecha, precio_coto, precio_carrefour, precio_dia
                  FROM precios_productos
                  ORDER BY producto, fecha DESC;'''
    precios = db.ejecutar_consulta(consulta)
    df_precios = pd.DataFrame(precios)
    # Supongamos que quieres cambiar los nombres de las columnas 'precio_coto', 'precio_carrefour' y 'precio_dia' a minúsculas
    nuevos_nombres = {'precio_coto': 'Coto', 'precio_carrefour': 'Carrefour', 'precio_dia': 'Día'}
    # Utiliza el método rename() para cambiar los nombres de las columnas
    df_precios = df_precios.rename(columns=nuevos_nombres)
    df_precios = df_precios(index_)
except Exception as e:
    print("Error al consultar los precios de los productos:", e)

df_precios

Conexión cerrada correctamente.


,producto,fecha,Coto,Carrefour,Día
0,"Aceite de Girasol Natura 1,5 Lts",2024-04-06,2808.00,2805.00,2805.00
1,Arroz Integral Molinos Ala 500 Gr,2024-04-06,1519.00,1350.00,1510.00
2,Azúcar Ledesma 1kg,2024-04-06,1399.00,1400.00,1395.00
3,Harina Blanca Flor,2024-04-06,809.00,609.00,709.00
4,Harina Leudante Pureza 1 Kg,2024-04-06,1118.00,1120.00,1170.00
5,Harina Morixe 0000 1kg,2024-04-06,870.00,609.00,870.00
6,Polenta Blanca Flor,2024-04-06,809.00,None,709.00
7,Yerba Mate Playadito Suave 1 Kg,2024-04-06,4865.00,4105.00,3650.00


In [4]:
import requests
from bs4 import BeautifulSoup
import web_scrapping

def extraer_precio_coto(url):
    # Función para extraer el precio del producto desde la página web de Coto
    headers = {'User-Agent': 'Mozilla/5.0'}     # Se establece el User-Agent para evitar bloqueos por parte del servidor
    response = requests.get(url, headers=headers)       # Se realiza una solicitud GET a la URL proporcionada
    if response.status_code == 200:     # Se verifica si la solicitud fue exitosa (código de estado 200)
        soup = BeautifulSoup(response.text, 'html.parser')      # Se crea un objeto BeautifulSoup para analizar el contenido HTML de la página web
        precio_element = soup.find('span', class_='atg_store_newPrice')     # Se busca el elemento que contiene el precio del producto
        if precio_element:      # Si se encuentra el elemento del precio en la primera etiqueta
            return web_scrapping.limpiar_precio(precio_element.text.strip())
        else:       # Si no se encuentra el precio en la primera etiqueta, intentar encontrarlo en la segunda
            precio_descuento_element = soup.find('span', class_='price_discount')
            if precio_descuento_element:        # Si se encuentra el precio en la segunda etiqueta
                return web_scrapping.limpiar_precio(precio_descuento_element.text.strip())
            else:
                return None     # Si no se encuentra el elemento del precio en ninguna etiqueta, se devuelve None
    else:
        return "Error al acceder a la página"       # Si la solicitud no fue exitosa, se devuelve un mensaje de error

In [23]:
def limpiar_precio_desc_coto(precio_str):
    # Eliminar caracteres no numéricos y convertir el precio a formato decimal
    precio_str = precio_str.replace('$', '').replace(',', '')
    precio_float = float(precio_str)
    
    # Ajustar el formato decimal a dos decimales
    precio_decimal = round(precio_float, 2)
    
    return precio_decimal


In [25]:
import requests
from bs4 import BeautifulSoup
import web_scrapping

def extraer_precio_coto(url):
    headers = {'User-Agent': 'Mozilla/5.0'}  
    response = requests.get(url, headers=headers)  
    if response.status_code == 200:  
        soup = BeautifulSoup(response.text, 'html.parser')  
        
        # Buscar precio en las etiquetas de descuento
        precio_descuento_element = soup.find('span', class_='price_discount')
        if precio_descuento_element:
            return limpiar_precio_desc_coto(precio_descuento_element.text.strip())
        
        precio_descuento_gde_element = soup.find('span', class_='price_discount_gde')
        if precio_descuento_gde_element:
            return limpiar_precio_desc_coto(precio_descuento_gde_element.text.strip())
        
        # Extraer precio de 'atg_store_newprice'
        atg_store_newprice = soup.find('span', class_='atg_store_newPrice')
        if atg_store_newprice:
            return web_scrapping.limpiar_precio(atg_store_newprice.text.strip())
                
    else:  
        return "Error al acceder a la página"



In [26]:
precio_coto = extraer_precio_coto('https://www.cotodigital3.com.ar/sitios/cdigi/producto/-galletitas-oreo-rellenas-con-crema-sabor-original-1825g-menos-sodio/_/A-00507347-00507347-200')
precio_coto

1142.04

In [27]:
precio_coto = extraer_precio_coto('https://www.cotodigital3.com.ar/sitios/cdigi/producto/-galletitas-oreo-rellenas-con-crema-sabor-original-118g-menos-sodio/_/A-00532297-00532297-200')
precio_coto

698.09

In [28]:
precio_leche = extraer_precio_coto('https://www.cotodigital3.com.ar/sitios/cdigi/producto/-leche-e-polvo-desc-fortc-v-la-serenisi-paq-200-grm/_/A-00510583-00510583-200')
precio_leche

1799.0